## Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import unicodedata
import os
import yfinance as yf
import math
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from matplotlib.dates import DateFormatter
import statistics
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder,label_binarize
from textblob import TextBlob, Word
from datetime import datetime, timedelta
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc

In [ ]:
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

In [ ]:
import os
work_directory = os.getcwd() 

In [ ]:
stock_tweet_path=work_directory+"/stock_tweets.csv"
stock_data_path=work_directory+"/stock_yfinance_data.csv"

In [ ]:
tickers = ['TSLA' ,'SMCI', 'AMZN', 'AVGO' ,'ALB' ,'MSFT' ,'META', 'NFLX' ,'NVDA']

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Setup Selenium WebDriver
service = Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Enable headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=service, options=chrome_options)


def last_11_months(time_text):
    if 'months' in time_text:
        months = int(time_text.split()[0])
        return months <= 11
    return True

# Initialize an empty DataFrame to collect all news
head1 = pd.DataFrame()

for ticker in tickers:
    url = f'https://finance.yahoo.com/quote/{ticker}/news/'
    driver.get(url)
    time.sleep(4)
    print(f'Collecting news related to {ticker}')
    previous_height = driver.execute_script('return document.body.scrollHeight')
    count = 0
    news = []
    
    while True:
        content_divs = driver.find_elements(By.CSS_SELECTOR, 'div.content.svelte-w27v8j')
        
        latest_time_text = ''
        for content_div in content_divs:
            try:
                a_tag = content_div.find_element(By.CSS_SELECTOR, 'a.subtle-link.fin-size-small.titles.noUnderline.svelte-wdkn18')
                title = a_tag.get_attribute('title') if a_tag else None
                
                publishing_div = content_div.find_element(By.CSS_SELECTOR, 'div.publishing.font-condensed.svelte-1k3af9g')
                time_text = publishing_div.text.split('•')[-1].strip() if publishing_div else None
                latest_time_text = time_text
                if latest_time_text and last_12_months(latest_time_text):
                    news.append({'Date': time_text, 'Headline': title, 'Stock Name': ticker})
            except Exception as e:
                print(f"Error extracting data: {e}")

        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(4)
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == previous_height:
            count += 1
            if count > 4:
                break
        else:
            previous_height = new_height
            count = 0
        
        if latest_time_text and not last_11_months(latest_time_text):
            break
    
    df_news = pd.DataFrame(news)
    
    if not df_news.empty:
        print(f'Starting news related to {ticker} is {df_news["Date"].iloc[0]}')
        print(f'Ending news related to {ticker} is {df_news["Date"].iloc[-1]}')
        print('Dataset size is ', df_news.shape[0])
        head1 = pd.concat([head1, df_news], ignore_index=True)

# Close the WebDriver
driver.quit()

In [ ]:
# Function to convert relative time text to actual date
def convert_to_date(time_text):
    now = datetime.now()
    if time_text == 'yesterday':
        return now - timedelta(days=1)
    if time_text == 'last month':
        return now - timedelta(days=30)
    if 'minute' in time_text:
        minutes = int(time_text.split()[0])
        return now - timedelta(minutes=minutes)
    elif 'hour' in time_text:
        hours = int(time_text.split()[0])
        return now - timedelta(hours=hours)
    elif 'day' in time_text:
        days = int(time_text.split()[0])
        return now - timedelta(days=days)
    elif 'week' in time_text:
        weeks = int(time_text.split()[0])
        return now - timedelta(weeks=weeks)
    elif 'month' in time_text:
        months = int(time_text.split()[0])
        return now - timedelta(days=30*months)
    elif 'today' in time_text.lower():
        return now
    else:
        return None
head1['Date'] = head1['Date'].apply(convert_to_date)

In [ ]:
import requests
import time

def fetch_news(year, month, api_key):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        articles = data['response']['docs']
        return articles
    else:
        print(f"Failed to fetch data for {year}-{month}: {response.status_code}")
        return []

# Initialize lists for headlines and dates
all_headlines = []
all_dates = []

# API key
api_key = 'Z7LiuLGnozS868StkM3QcbQhsJA8acDt'

# Fetch data for each year from 2010 to 2024
for year in range(2020, 2024):
    for month in range(1, 13):
        print(f"Fetching data for {year}-{month}")
        try:
            articles = fetch_news(year, month, api_key)
            if articles:
                all_headlines.extend([article['headline']['main'] for article in articles])
                all_dates.extend([article['pub_date'] for article in articles])
            # Sleep to respect API rate limits
            time.sleep(6)  # Adjust sleep time as needed based on API rate limits
        except Exception as e:
            print(f"Error fetching data for {year}-{month}: {e}")


 


In [ ]:
# Create a DataFrame
df = pd.DataFrame({'Date': all_dates, 'Headline': all_headlines})

# Convert datetime column to datetime format
df['Date'] = pd.to_datetime(df['Date'])
df['Stock Name']='NVDA'
print('data extracted')

In [ ]:
head1 = pd.concat([head1, df], ignore_index=True)


In [ ]:
# Save the collected news to a CSV file
head1.to_csv(stock_tweet_path, index=False)


## Related stock data

In [ ]:
Stock=pd.DataFrame()  # Example ticker symbol for Apple
start_date = "2023-07-01"  # Specify start date
end_date = "2024-06-14"   
dfs = [] 
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date).reset_index()  # Reset index to make 'Date' a column
    data['Stock Name'] = ticker  # Add a column for the ticker symbol
    dfs.append(data)
data=yf.download('NVDA',start='2020-01-01',end='2023-12-30').reset_index()
data['Stock Name']='NVDA'
dfs.append(data)
# Combine all individual dataframes into a single dataframe
Stock = pd.concat(dfs, ignore_index=True)

# Reorder columns as per the desired format
Stock = Stock[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock Name']]

 

In [ ]:
Stock.head()

In [ ]:
Stock['Stock Name'].value_counts()

In [ ]:
Stock.shape

In [ ]:
Stock.to_csv(stock_data_path, index=False)

## Data

In [ ]:
head1=pd.read_csv(stock_tweet_path)

In [ ]:
print(head1['Stock Name'].unique())

In [ ]:
head1.shape

## Sentiment analysis

In [ ]:
head1['polarity_score']=''
head1['Headline'] = head1['Headline'].astype(str)
head1.head()

In [ ]:
sent_analyze = SentimentIntensityAnalyzer()

# Analyze each tweet and store the compound score
for ind, row in head1.iterrows():
    headline = row["Headline"]
    sentence_i = unicodedata.normalize("NFKD", headline)
    sent_sent = sent_analyze.polarity_scores(sentence_i)
    head1.at[ind, "polarity_score"] = sent_sent["compound"]




In [ ]:
head1.head()

In [ ]:
head1["sentiment_label"] = head1["Headline"].apply(lambda x: "pos" if sent_analyze.polarity_scores(x)["compound"] > 0 else ("neu" if sent_analyze.polarity_scores(x)["compound"] ==0 else "neg"))

In [ ]:
head1.head()

In [ ]:
head1["sentiment_label"].value_counts()

In [ ]:
head1["sentiment_label"] = LabelEncoder().fit_transform(head1["sentiment_label"])

In [ ]:
head1['Date'] = pd.to_datetime(head1['Date'], errors='coerce', utc=True)
head1["Date"] = head1["Date"].dt.date
head1.head()

In [ ]:
head1["sentiment_label"].value_counts()

In [ ]:
head1.shape

In [ ]:
# Preprocessing function
def preprocessing(df):
    # Convert to lower case
    df = df.str.lower()
    # Remove punctuation
    df = df.str.replace('[^\w\s]', '', regex=True)
    # Remove numbers
    df = df.str.replace('\d', '', regex=True)
    # Remove stopwords
    sw = stopwords.words('english')
    df = df.apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
    # Remove infrequent words
    temp_df = pd.Series(' '.join(df).split()).value_counts()
    drops = temp_df[temp_df <= 1]
    df = df.apply(lambda x: " ".join(x for x in x.split() if x not in drops))
    return df

# Tokenization function
def tokenization(df):
    df = df.apply(lambda x: TextBlob(x).words)
    return df

# Lemmatization function
def lemmatized(df):
    df = df.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    return df

# Term frequency function
def term_fre(df):
    tf = df.apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()
    tf.columns = ['words', 'tf']
    tf_sorted = tf.sort_values(by='tf', ascending=False)
    return tf_sorted

# TF-IDF function
def tfidf(X, fit_vectorizer=None):
    if fit_vectorizer is None:
        fit_vectorizer = TfidfVectorizer()
        X_tf_idf_word = fit_vectorizer.fit_transform(X)
    else:
        X_tf_idf_word = fit_vectorizer.transform(X)
    return X_tf_idf_word, fit_vectorizer

# Function to preprocess and vectorize
def givefinalX(X, vectorizer=None):
    # Apply preprocessing
    X = preprocessing(X)

    # Apply tokenization (if needed)
    # X = tokenization(X)

    # Apply lemmatization
    X = lemmatized(X)

    # Convert to TF-IDF
    X_tf_idf_word, vectorizer = tfidf(X, vectorizer)
    return X_tf_idf_word, vectorizer


# Split the data
X = head1['Headline']
y = head1['sentiment_label']
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## Sentiment modelling

In [ ]:
# Convert to TF-IDF
X_tf_idf_word_train, vectorizer = givefinalX(X_train)

X_tf_idf_word_test, _ = givefinalX(X_test, vectorizer)

# Train RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_tf_idf_word_train, y_train)


In [ ]:
 # Cross-validation score
cv_score = cross_val_score(rf_model, X_tf_idf_word_train, y_train, cv=5, n_jobs=-1).mean()
print(f"Cross-validation score: {cv_score}")


In [ ]:
 # Predictions on test set
y_pred = rf_model.predict(X_tf_idf_word_test)
y_prob = rf_model.predict_proba(X_tf_idf_word_test)
print(f"Test set predictions: {y_pred}")


In [ ]:
# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
roc_auc = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2]), y_prob, average='macro', multi_class='ovr')

# Print evaluation metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'ROC AUC: {roc_auc}')


In [ ]:
# ROC Curve for each class
fpr = {}
tpr = {}
roc_auc = {}

# Binarize the output
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
n_classes = y_test_bin.shape[1]

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['blue', 'red', 'green']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Multi-Class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Evaluate on the test set
test_score = rf_model.score(X_tf_idf_word_test, y_test)
print(f"Test set score: {test_score}")

## Stock prediction

In [ ]:
df = head1[["Date","polarity_score","Stock Name"]]
 

In [ ]:
df.shape

In [ ]:
df["Stock Name"].value_counts()

In [ ]:
# Group by "Date" and "Stock Name" and calculate the mean of "sentence_score"
df_grouped = df.groupby(["Date", "Stock Name"]).mean().reset_index()

In [ ]:
print(df_grouped.shape)

In [ ]:
df_grouped.head()

In [ ]:
df_grouped['Stock Name'].value_counts()

In [ ]:
df_grouped.columns

In [ ]:
Stock=pd.read_csv(stock_data_path)

In [ ]:
print(Stock.shape)
Stock.head()

In [ ]:
Stock['Stock Name'].value_counts()

In [ ]:
Stock['Stock Name'].unique()

In [ ]:
Stock["Date"]=pd.to_datetime(Stock["Date"])
Stock["Date"]=Stock["Date"].dt.date
print(Stock.shape)
Stock.head()

## Let's See affect of tweet on a specific stock

In [ ]:
stk_name='NVDA'

In [ ]:
stock = Stock[Stock["Stock Name"] == stk_name]
stock["Date"] = pd.to_datetime(stock["Date"])
stock["Date"] = stock["Date"].dt.date
# Reset the index of appl_stock
stock.reset_index(drop=True, inplace=True)
stock.head()

In [ ]:
stock.shape

In [ ]:
# Now filter the grouped DataFrame for rows
df = df_grouped[df_grouped['Stock Name'] == stk_name]
# Reset the index of appl_df
df.reset_index(drop=True, inplace=True)

In [ ]:
print(df.shape)
df.head()

In [ ]:
# Now perform the join operation
df = stock.join(df.set_index(["Date", "Stock Name"]), on=["Date", "Stock Name"], how="inner")
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
fig,ax = plt.subplots(figsize=(15,8))
ax.plot(df["Date"], df["Adj Close"])
ax.set(xlabel="Date",ylabel="USD",title=f"{stk_name} Stock Price")
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(15,8))
ax.plot(df["Date"], df["polarity_score"])
ax.set(xlabel="Date",ylabel="polarity Score",title=f"{stk_name}'s Public Sentiment")
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
plt.show()

In [ ]:
# fin_df = appl_df[["Date","Close","sentence_score"]]
fin_df=df
# fin_df = fin_df.reset_index(drop=True)
print(fin_df.shape)
fin_df.head()

In [ ]:
def SentimentAdjustedMA(df, ma_days):
    def weight_multiplier(close,sent_score):
        maxi = max(close)
        mini = min(close)
        len_close = len(close)
        if len_close<2:
            interval = math.sqrt(close[0])
        else:
            interval = statistics.variance(close)
        max_var = interval
        weighted=0
        for i in range(len_close):
            if sent_score[i] < 0:
                weighted += close[i] + (2*sent_score[i]*max_var)
            else:
                weighted += close[i] + (sent_score[i]*max_var)
        return weighted/len_close-1
    samas = [df.loc[0,"Adj Close"]]
    rows = df.shape[0]
    for i in range(1,rows):
        if i < ma_days:
            mini_df = df.iloc[:i+1,:]
        elif i + ma_days >= rows:
            mini_df = df.iloc[i:,:]
        else:
            mini_df = df.iloc[i-ma_days+1:i+1,:]
        sama_value = weight_multiplier(mini_df["Adj Close"].tolist(),mini_df["polarity_score"].tolist())
        samas.append(sama_value)
    return samas

In [ ]:
sma5= SentimentAdjustedMA(fin_df, ma_days=5)
fin_df["SMA(5)"] = sma5
sma20= SentimentAdjustedMA(fin_df, ma_days=10)
fin_df["SMA(20)"] = sma20
fin_df.head()

In [ ]:
fig,ax = plt.subplots(figsize=(15,8))
ax.plot(fin_df["Date"], fin_df["SMA(5)"], label="SMA(5)", linestyle="--", color="g")
ax.plot(fin_df["Date"], fin_df["Adj Close"], label="Original", color="b")
ax.set(xlabel="Date",ylabel="USD",title=f"{stk_name} Stock Price")
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
plt.legend()
plt.show()

In [ ]:
# Sort the dataframe by the 'Date' column
fin_df = fin_df.sort_values(by='Date')

# Reset the index to keep the dataframe tidy
fin_df = fin_df.reset_index(drop=True)

In [ ]:
# Generate signals based on polarity score and additional filter (SMA)
def generate_signals(df):
    df['SMA'] = df['Close'].rolling(window=20).mean()  # 20-day simple moving average
    df['Signal'] = 0
    df['Signal'] = np.where((df['polarity_score'] > 0) & (df['Close'] > df['SMA']), 1, df['Signal'])
    df['Signal'] = np.where((df['polarity_score'] < 0) & (df['Close'] < df['SMA']), -1, df['Signal'])
    df['Order'] = df['Signal'].diff()
    return df

# Calculate returns and performance metrics with stop-loss, take-profit, and position sizing
def calculate_performance(df, initial_cash=1000, stop_loss_pct=0.10, take_profit_pct=0.20, position_size_pct=0.20):
    cash = initial_cash
    position = 0
    purchase_price = 0
    portfolio_values = []
    
    for index, row in df.iterrows():
        if row['Order'] == -1 and cash > 0:  # Buy signal
            position = (cash * position_size_pct) / row['Close']
            cash += position * row['Close']
            purchase_price = row['Close']
        elif row['Order'] == 1 and position > 0:  # Sell signal
            cash -= position * row['Close']
            position = 0
        elif position > 0:
            if row['Close'] < purchase_price * (1 - stop_loss_pct):  # Stop-loss condition
                cash += position * row['Close']
                position = 0
                df.loc[index, 'Order'] = -1  # Mark this as a sell due to stop-loss
            elif row['Close'] > purchase_price * (1 + take_profit_pct):  # Take-profit condition
                cash += position * row['Close']
                position = 0
                df.loc[index, 'Order'] = -1  # Mark this as a sell due to take-profit

        portfolio_value = cash + position * row['Close']
        portfolio_values.append(portfolio_value)

    df['Portfolio Value'] = portfolio_values
    final_portfolio_value = portfolio_values[-1]
    returns = df['Portfolio Value'].pct_change().dropna()
    sharpe_ratio = (252**0.5) * returns.mean() / returns.std()  # Assuming 252 trading days in a year
    total_trades = len(df[df['Order'].abs() == 1])
    winning_trades = len(df[(df['Order'] == -1) & (df['Close'] > df['Close'].shift(1))])
    win_ratio = winning_trades / total_trades if total_trades > 0 else 0

    performance = {
        "Final Portfolio Value": final_portfolio_value,
        "Sharpe Ratio": sharpe_ratio,
        "Number of Trades Executed": total_trades,
        "Win Ratio": win_ratio
    }

    return performance

# Load your dataframe
# Ensure that the 'Date' column is in datetime format and sort by date
fin_df['Date'] = pd.to_datetime(fin_df['Date'])
fin_df = fin_df.sort_values(by='Date')

# Generate signals and calculate performance
fin_df = generate_signals(fin_df)
performance = calculate_performance(fin_df)

# Print performance metrics
print(f"Final Portfolio Value: ${performance['Final Portfolio Value']:.2f}")
print(f"Sharpe Ratio: {performance['Sharpe Ratio']:.2f}")
print(f"Number of Trades Executed: {performance['Number of Trades Executed']}")
print(f"Win Ratio: {performance['Win Ratio']:.2f}")

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(fin_df['Date'], fin_df['Close'], label='Closing Price')

# Plot buy signals
buy_signals = fin_df[fin_df['Order'] == 1]
plt.plot(buy_signals['Date'], buy_signals['Close'], '^', markersize=10, color='g', label='Buy Signal')

# Plot sell signals
sell_signals = fin_df[fin_df['Order'] == -1]
plt.plot(sell_signals['Date'], sell_signals['Close'], 'v', markersize=10, color='r', label='Sell Signal')

plt.title('Closing Price and Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Plot portfolio value
plt.figure(figsize=(12, 6))
plt.plot(fin_df['Date'], fin_df['Portfolio Value'], label='Portfolio Value')
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.show()

final_path=work_directory+"/signals.csv"
fin_df.to_csv('final_path', index=False)  # Adjust the path to your desired output file location




In [ ]:
# Plot buy signals
buy_signals = fin_df[fin_df['Order'] == 1]
plt.scatter(buy_signals['Date'], buy_signals['Close'], marker='^', color='green', label='Buy Signal', alpha=1)

# Plot sell signals
sell_signals = fin_df[fin_df['Order'] == -1]
plt.scatter(sell_signals['Date'], sell_signals['Close'], marker='v', color='red', label='Sell Signal', alpha=1)

plt.title('Closing Price and Buy/Sell Signals')
plt.xlabel('Date')


plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plotting the Portfolio Value over Time
plt.figure(figsize=(14, 7))
plt.plot(fin_df['Date'], fin_df['Portfolio Value'], label='Portfolio Value', color='purple')
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid()
plt.show()

# Print performance metrics
print(f"Final Portfolio Value: ${performance['Final Portfolio Value']:.2f}")
print(f"Sharpe Ratio: {performance['Sharpe Ratio']:.2f}")
print(f"Number of Trades Executed: {performance['Number of Trades Executed']}")
print(f"Win Ratio: {performance['Win Ratio']:.2f}")